# IBD Analysis

## Introduction



The following code will merge 5 cleaned files and perform data preprocessing

In [ ]:
#Importing required packages

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np

In [ ]:
#Loading all cleaned datasets

# Load dataset 1
exam_notes_df = pd.read_csv('IBD Clinical Notes Data')
# Load dataset 2
image_CD_df = pd.read_csv("IBD Image Data for Crohn's Disease ")
# Load dataset 3
image_UC_df = pd.read_csv('IBD Image Data for Ulcerative Colitis')

# Load dataset 3
GI_chrons_df = pd.read_csv("GI Responses for Crohn Disease")

# Load dataset 4
GI_ulcerative_df = pd.read_csv("GI Responses for Ulcerative Colitis")

# Data Preparation And Cleaning

## Merging Exam Notes and Image UC and CD 


In [ ]:
# Merging Exam Notes and Image UC and CD
merged_df1 = pd.merge(exam_notes_df, image_CD_df, on='Exam_ID', how='outer')
merged_df1 = pd.merge(merged_df1, image_UC_df, on='Exam_ID', how='outer')

merged_df1['Image_Info_x'] = merged_df1['Image_Info_x'].astype(str)
merged_df1['Image_Info_y'] = merged_df1['Image_Info_y'].astype(str)

merged_df1['Image_Info'] = merged_df1['Image_Info_x'].fillna('') + merged_df1['Image_Info_y'].fillna('')
merged_df1 = merged_df1.drop(['Image_Info_x', 'Image_Info_y'], axis=1)

In [ ]:
merged_df1.shape

In [ ]:
len(merged_df1['Exam_ID'].unique())

In [ ]:
def print_null_percentage(df):
    for col in df.columns:
        null_count = df[col].isnull().sum()
        null_percentage = (null_count / len(df)) * 100
        print(f"Percentage of null values in {col}: {null_percentage}%")

In [ ]:
print_null_percentage(merged_df1)

## Merged Exam notes and Images dataframe + GI Ulcerative dataframe

Total 361
Active = 170
Inactive = 191

In [ ]:
# Merging Exam Notes and Image UC and CD and GI Ulcerative dataframe 
merged_df2 = pd.merge(merged_df1, GI_ulcerative_df, on='Exam_ID',how='right')

In [ ]:
merged_df2

In [ ]:
merged_df2= merged_df2.dropna()

In [ ]:
print_null_percentage(merged_df2)

In [ ]:
merged_df2.shape

In [ ]:
merged_df2_Inactive = merged_df2[merged_df2['Disease_Severity'] != 'Active']

In [ ]:
merged_df2_Inactive.shape

## Merged Exam notes and Images dataframe + GI Chrons dataframe

Total 372
Active = 200
Inactive = 172

In [ ]:
# Merging Exam Notes and Image UC and CD and GI Chrons dataframe
merged_df3 = pd.merge(merged_df1, GI_chrons_df, on='Exam_ID',how='right')

In [ ]:
merged_df3

In [ ]:
merged_df3= merged_df3.dropna()

In [ ]:
print_null_percentage(merged_df3)

In [ ]:
merged_df3.shape

In [ ]:
merged_df3_Inactive = merged_df3[merged_df3['Disease_Severity'] != 'Active']

In [ ]:
merged_df3_Inactive.shape

In [ ]:
merged_df3.loc[merged_df3['Exam_ID'] == '3908FC01091A445B88E2D241A85BA442']

## Find common Exam IDs in these two merged dataframes

In [ ]:
def find_common_ids(df1, df2, primary_key_column):
    # Perform an inner merge based on the primary key column
    common_keys = df1.merge(df2, on=primary_key_column)[primary_key_column].tolist()

    # Return the common primary keys as a list
    return common_keys

In [ ]:
common_ids = find_common_ids(merged_df2,merged_df3,'Exam_ID')

In [ ]:
def drop_common_ids(df1, df2, common_ids):
    # Drop common keys from df1
    df1 = df1[~df1['Exam_ID'].isin(common_ids)].reset_index(drop=True)

    # Drop common keys from df2
    df2 = df2[~df2['Exam_ID'].isin(common_ids)].reset_index(drop=True)

In [ ]:
drop_common_ids(merged_df2,merged_df2,common_ids)

## Merging the two merged dataframes to get the final dataframe

In [ ]:
final_IBD_df = pd.merge(merged_df2, merged_df3, on='Exam_ID',how='outer')

In [ ]:
print_null_percentage(final_IBD_df)

In [ ]:
final_IBD_df.shape

#### Filtering the common data and will add this filtered common data after combining common columns in final mereged dataset

In [ ]:
filtered_final_IBD_df = final_IBD_df.dropna(how='any')

In [ ]:
filtered_final_IBD_df

In [ ]:
filtered_final_IBD_df = filtered_final_IBD_df.drop(['Disease_Severity_y', 'Exam_Notes_y','Image_Info_y','Ulcer_y'], axis=1)

In [ ]:
filtered_final_IBD_df = filtered_final_IBD_df.rename(columns={"Disease_Severity_x": "Disease_Severity", 
                                                    "Exam_Notes_x": "Exam_Notes",
                                                    "Image_Info_x": "Image_Info",
                                                    "Ulcer_x": "Ulcer"})


In [ ]:
filtered_df1 = final_IBD_df[~final_IBD_df['Exam_ID'].isin(filtered_final_IBD_df['Exam_ID'])]

In [ ]:
filtered_df1.shape

In [ ]:
print_null_percentage(filtered_df1)

In [ ]:
filtered_df1.head()

In [ ]:
filtered_df1_copy = filtered_df1.copy()
filtered_df1_copy['Image_Info_x'] = filtered_df1_copy['Image_Info_x'].astype(str)
filtered_df1_copy['Image_Info_y'] = filtered_df1_copy['Image_Info_y'].astype(str)

In [ ]:
filtered_df1_copy.head()

In [ ]:
filtered_df1_copy['Disease_Severity'] = filtered_df1_copy['Disease_Severity_x'].fillna('') + filtered_df1_copy['Disease_Severity_y'].fillna('')
filtered_df1_copy['Ulcer'] = filtered_df1_copy['Ulcer_x'].fillna('') + filtered_df1_copy['Ulcer_y'].fillna('')
filtered_df1_copy['Exam_Notes'] = filtered_df1_copy['Exam_Notes_x'].fillna('') + filtered_df1_copy['Exam_Notes_y'].fillna('')
filtered_df1_copy['Image_Info'] = filtered_df1_copy['Image_Info_x'].fillna('') + filtered_df1_copy['Image_Info_y'].fillna('')

In [ ]:
filtered_df1_copy.drop(['Disease_Severity_x', 'Disease_Severity_y','Exam_Notes_x','Exam_Notes_y','Image_Info_x','Image_Info_y','Ulcer_x','Ulcer_y'], axis=1, inplace=True)

In [ ]:
print_null_percentage(filtered_df1_copy)

# Override - new change

In [ ]:
filtered_df1_copy['Image_Info'] = filtered_df1_copy['Image_Info'].replace('nannannan', np.nan)
null_values = filtered_df1_copy['Image_Info'].isna().any().any()
print(null_values)

In [ ]:
filtered_df1_copy = filtered_df1_copy.dropna()

----------------------------------------------------

In [ ]:
filtered_df1_copy.shape

## Merging the two final dataframes which has complete data and which has either GI CD or GI UC data

Total - 728 datapoints

In [ ]:
#final_IBD_df_new = filtered_df1_copy.append(filtered_final_IBD_df)
final_IBD_df_new = pd.concat([filtered_df1_copy, filtered_final_IBD_df], ignore_index=True)

In [ ]:
final_IBD_df_new.shape

In [ ]:
print_null_percentage(final_IBD_df_new)

In [ ]:
filtered_df1_copy['Image_Info'] = filtered_df1_copy['Image_Info'].astype(str)

In [ ]:
# Dropping rows with missing values
filtered_df1_copy = filtered_df1_copy.dropna()

In [ ]:
filtered_df1_copy.shape

# Data Preprocessing

In [ ]:
# Function to remove special characters
def remove_special_characters(df, column_name):
    # Define the regular expression pattern to match non-alphanumeric characters
    pattern = re.compile(r'[^a-zA-Z0-9\s]')
    # Remove special characters from every value in the specified column
    df[column_name] = df[column_name].str.replace(pattern, '')
    return df

In [ ]:
def remove_stopwords(df, column_name):
    # Get the list of stopwords
    stop_words = set(stopwords.words('english'))

    # Remove stopwords from every value in the specified column
    df[column_name] = df[column_name].apply(lambda x: ' '.join([word for word in str(x).split() if word.lower() not in stop_words]))

    # Remove duplicate words like "Item", "Text"
    df[column_name] = df[column_name].str.replace('Item', '').str.replace('Text', '').str.replace('Patient','').str.replace('patient','').str.replace('Procedure','').str.replace('procedure','').str.replace('nan','').str.replace('undefined','')

    # Remove empty elements from every value in the specified column
    df[column_name] = df[column_name].apply(lambda x: ' '.join([word for word in str(x).split() if word]))
    return df

In [ ]:
final_IBD_df_new = remove_special_characters(final_IBD_df_new,'Exam_Notes')
final_IBD_df_new = remove_stopwords(final_IBD_df_new,'Exam_Notes')
final_IBD_df_new = remove_special_characters(final_IBD_df_new,'Image_Info')
final_IBD_df_new['Image_Info'] = final_IBD_df_new['Image_Info'].astype(str)
final_IBD_df_new = remove_stopwords(final_IBD_df_new,'Image_Info')

In [ ]:
final_IBD_df_new.shape

In [ ]:
print_null_percentage(final_IBD_df_new)

In [ ]:
final_IBD_df_new['Exam_Notes'] = final_IBD_df_new['Exam_Notes'].apply(lambda x: 'undefined' if all(word == 'undefined' for word in str(x).split()) else x)

In [ ]:
# Specify the file path for the Excel file
Merged_File = 'Merged_File'

# Write the DataFrame to the CSV file
final_IBD_df_new.to_csv(Merged_File, index=False)